Import packages

In [2]:
import sqlite3
import pandas as pd
import os
import Code.Preparation as prep

### Clean and combine happiness reports dataset

Load datasets

In [322]:
df2015 = pd.read_csv(prep.path('happy/2015.csv'))
df2015['Year'] = (2015)
df2016 = pd.read_csv(prep.path('happy/2016.csv'))
df2016['Year'] = 2016
df2017 = pd.read_csv(prep.path('happy/2017.csv'))
df2017['Year'] = 2017
df2018 = pd.read_csv(prep.path('happy/2018.csv'))
df2018['Year'] = 2018
df2019 = pd.read_csv(prep.path('happy/2019.csv'))
df2019['Year'] = 2019

In [442]:
happydf = pd.merge(df2015, df2016, how='outer').merge(df2017, how='outer').merge(df2018, how='outer').merge(df2019, how='outer')

In [443]:
happydf.Country = happydf.Country.fillna(happydf['Country or region'])
happydf['Happiness Rank'] = happydf['Happiness Rank'].fillna(happydf['Happiness.Rank'])
happydf['Happiness Rank'] = happydf['Happiness Rank'].fillna(happydf['Overall rank'])
happydf['Happiness Score'] = happydf['Happiness Score'].fillna(happydf['Happiness.Score'])
happydf['Happiness Score'] = happydf['Happiness Score'].fillna(happydf['Score'])
happydf['Economy (GDP per Capita)'] = happydf['Economy (GDP per Capita)'].fillna(happydf['Economy..GDP.per.Capita.'])
happydf['Economy (GDP per Capita)'] = happydf['Economy (GDP per Capita)'].fillna(happydf['GDP per capita'])
happydf['Health (Life Expectancy)'] = happydf['Health (Life Expectancy)'].fillna(happydf['Health..Life.Expectancy.'])
happydf['Health (Life Expectancy)'] = happydf['Health (Life Expectancy)'].fillna(happydf['Healthy life expectancy'])
happydf['Trust (Government Corruption)'] = happydf['Trust (Government Corruption)'].fillna(happydf['Perceptions of corruption'])
happydf['Trust (Government Corruption)'] = happydf['Trust (Government Corruption)'].fillna(happydf['Trust..Government.Corruption.'])
happydf['Freedom'] = happydf['Freedom'].fillna(happydf['Freedom to make life choices'])
happydf['Social support'] = happydf['Social support'].fillna(happydf['Family'])
happydf = pd.DataFrame(happydf[['Country', 'Year', 'Happiness Rank', 'Happiness Score', 'Economy (GDP per Capita)', 'Social support', 'Trust (Government Corruption)', 'Generosity']])

In [444]:
happydf.dropna(inplace=True)
happydf['Year'] = happydf['Year'].astype(str)
happydf['Year'] = pd.to_datetime(happydf['Year'])
# happydf = happydf.set_index(['Country', 'Year'])

In [446]:
happydf.to_csv('Data/CleanedHappy.csv')

In [447]:
ratedf = pd.read_csv(prep.path('sucide.csv'))
ratedf = ratedf.rename(columns = {'country':'Country', 'year':'Year'})
ratedf['Year'] = ratedf['Year'].astype(str)
ratedf['Year'] = pd.to_datetime(ratedf['Year'])
ratedf = ratedf.drop(columns=['country-year'])
# ratedf = ratedf.set_index(['Country', 'Year'])

In [479]:
ratedf.to_csv('Data/CleanedSuicide.csv')

In [448]:
conn = sqlite3.connect(prep.path('/World_Development/database.sqlite'))
cur = conn.cursor()
pd.read_sql("""
SELECT *
FROM sqlite_master
WHERE type='table';
""", conn)

,type,name,tbl_name,rootpage,sql
0,table,Country,Country,2,"CREATE TABLE Country (\n CountryCode TEXT,\..."
1,table,CountryNotes,CountryNotes,186,CREATE TABLE CountryNotes (\n Countrycode T...
2,table,Series,Series,948,"CREATE TABLE Series (\n SeriesCode TEXT,\n ..."
3,table,Indicators,Indicators,4448,CREATE TABLE Indicators (\n CountryName TEX...
4,table,SeriesNotes,SeriesNotes,1317550,CREATE TABLE SeriesNotes (\n Seriescode TEX...
5,table,Footnotes,Footnotes,1317587,CREATE TABLE Footnotes (\n Countrycode TEXT...


In [449]:
cur.execute("""
SELECT IndicatorName
FROM Indicators
UNION
SELECT IndicatorName
FROM Indicators
""")
cur.fetchall()

[('2005 PPP conversion factor, GDP (LCU per international $)',),
 ('2005 PPP conversion factor, private consumption (LCU per international $)',),
 ('ARI treatment (% of children under 5 taken to a health provider)',),
 ('Access to electricity (% of population)',),
 ('Access to electricity, rural (% of rural population)',),
 ('Access to electricity, urban (% of urban population)',),
 ('Access to non-solid fuel (% of population)',),
 ('Access to non-solid fuel, rural (% of rural population)',),
 ('Access to non-solid fuel, urban (% of urban population)',),
 ('Adequacy of social insurance programs (% of total welfare of beneficiary households)',),
 ('Adequacy of social protection and labor programs (% of total welfare of beneficiary households)',),
 ('Adequacy of social safety net programs (% of total welfare of beneficiary households)',),
 ('Adequacy of unemployment benefits and ALMP (% of total welfare of beneficiary households)',),
 ('Adjusted net enrolment rate, primary, both sexes (%

In [450]:
accessdf = pd.read_sql("""
SELECT *
FROM Indicators
WHERE (IndicatorName ='GDP per capita (current US$)' 
OR IndicatorName = 'GDP growth (annual %)' 
OR IndicatorName = 'GDP per capita growth (annual %)'
OR IndicatorName = 'Household final consumption expenditure per capita growth (annual %)'
OR IndicatorName = 'Life expectancy at birth, total (years)'
OR IndicatorName = 'Multilateral debt (% of total external debt)'
OR IndicatorName = 'Access to electricity (% of population)'
OR IndicatorName = 'Access to electricity, rural (% of rural population)'
OR IndicatorName = 'Access to electricity, urban (% of urban population)'
OR IndicatorName = 'Depth of the food deficit (kilocalories per person per day)'
OR IndicatorName = 'Fixed broadband subscriptions (per 100 people)'
OR IndicatorName = 'Fixed telephone subscriptions (per 100 people)'
OR IndicatorName = 'Mobile cellular subscriptions (per 100 people)'
OR IndicatorName = 'Improved sanitation facilities (% of population with access)'
OR IndicatorName = 'Improved sanitation facilities, rural (% of rural population with access)'
OR IndicatorName = 'Improved sanitation facilities, urban (% of urban population with access)'
OR IndicatorName = 'Improved water source (% of population with access)'
OR IndicatorName = 'Improved water source, rural (% of rural population with access)'
OR IndicatorName = 'Improved water source, urban (% of urban population with access)'
OR IndicatorName = 'Adequacy of social insurance programs (% of total welfare of beneficiary households)'
OR IndicatorName = 'Adequacy of social protection and labor programs (% of total welfare of beneficiary households)'
OR IndicatorName = 'Adequacy of social safety net programs (% of total welfare of beneficiary households)'
OR IndicatorName = 'Adequacy of unemployment benefits and ALMP (% of total welfare of beneficiary households)'
OR IndicatorName = 'Antiretroviral therapy coverage (% of people living with HIV)'
OR IndicatorName = 'Adjusted net national income per capita (annual % growth)'
OR IndicatorName = 'Annualized average growth rate in per capita real survey mean consumption or income, bottom 40% of population (%)'
OR IndicatorName = 'Annualized average growth rate in per capita real survey mean consumption or income, total population (%)'
OR IndicatorName = 'Employers, total (% of employment)'
OR IndicatorName = 'Labor force participation rate, total (% of total population ages 15+) (national estimate)'
OR IndicatorName = 'Labor force with primary education (% of total)'
OR IndicatorName = 'Labor force with secondary education (% of total)'
OR IndicatorName = 'Labor force with tertiary education (% of total)'
OR IndicatorName = 'Enrolment in primary education, both sexes (number)'
OR IndicatorName = 'Enrolment in secondary education, both sexes (number)'
OR IndicatorName = 'Enrolment in secondary general, both sexes (number)'
OR IndicatorName = 'Enrolment in secondary vocational, both sexes (number)'
OR IndicatorName = 'Out-of-school children of primary school age, both sexes (number)'
OR IndicatorName = 'Exports of goods and services (% of GDP)'
OR IndicatorName = 'Exports of goods and services (annual % growth)'
OR IndicatorName = 'Food exports (% of merchandise exports)'
OR IndicatorName = 'Food imports (% of merchandise imports)'
OR IndicatorName = 'Cost to export (US$ per container)'
OR IndicatorName = 'Cost to import (US$ per container)'
OR IndicatorName = 'High-technology exports (% of manufactured exports)'
OR IndicatorName = 'Imports of goods and services (% of GDP)'
OR IndicatorName = 'Imports of goods and services (annual % growth)')
""", conn)

In [451]:
accessdf.drop(columns= ['CountryCode', 'IndicatorCode'], inplace=True)
accessdf.rename(columns={'CountryName': 'Country'}, inplace=True)
accessdf['Year'] = accessdf['Year'].astype(str)
accessdf['Year'] = pd.to_datetime(accessdf['Year'])
# accessdf = accessdf.set_index(['Country', 'Year'])

In [452]:
datadf = pd.read_csv(prep.path('WDI/WDIData.csv'))

In [453]:
list(datadf['Indicator Name'])

['2005 PPP conversion factor, GDP (LCU per international $)',
 '2005 PPP conversion factor, private consumption (LCU per international $)',
 'Access to clean fuels and technologies for cooking (% of population)',
 'Access to electricity (% of population)',
 'Access to electricity, rural (% of rural population)',
 'Access to electricity, urban (% of urban population)',
 'Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+)',
 'Account ownership at a financial institution or with a mobile-money-service provider, female (% of population ages 15+)',
 'Account ownership at a financial institution or with a mobile-money-service provider, male (% of population ages 15+)',
 'Account ownership at a financial institution or with a mobile-money-service provider, older adults (% of population ages 25+)',
 'Account ownership at a financial institution or with a mobile-money-service provider, poorest 40% (% of population ages 15+)',
 'Accoun

In [454]:
datadf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422136 entries, 0 to 422135
Data columns (total 64 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Country Name    422136 non-null  object 
 1   Country Code    422136 non-null  object 
 2   Indicator Name  422136 non-null  object 
 3   Indicator Code  422136 non-null  object 
 4   1960            38296 non-null   float64
 5   1961            42017 non-null   float64
 6   1962            44198 non-null   float64
 7   1963            44084 non-null   float64
 8   1964            44611 non-null   float64
 9   1965            47305 non-null   float64
 10  1966            46902 non-null   float64
 11  1967            48633 non-null   float64
 12  1968            48209 non-null   float64
 13  1969            50011 non-null   float64
 14  1970            92892 non-null   float64
 15  1971            99126 non-null   float64
 16  1972            101898 non-null  float64
 17  1973      

In [456]:
worlddf = pd.DataFrame(datadf.groupby(['Country Name','Indicator Name']).mean().stack())
worlddf = worlddf.reset_index()
worlddf = worlddf.rename(columns={0: 'Value', 'level_2': 'Year', 'Country Name': 'Country', 'Indicator Name': 'IndicatorName'})
worlddf['Year'] = worlddf['Year'].astype(str)
worlddf['Year'] = pd.to_datetime(worlddf['Year'])

In [457]:
worlddf = worlddf[(worlddf.IndicatorName == 'Imports of goods and services (annual % growth)')| 
        (worlddf.IndicatorName == 'Imports of goods and services (% of GDP)')| 
        (worlddf.IndicatorName == 'High-technology exports (% of manufactured exports)')| 
        (worlddf.IndicatorName == 'Cost to import (US$ per container)')| 
        (worlddf.IndicatorName == 'Cost to export (US$ per container)')| 
        (worlddf.IndicatorName == 'Food imports (% of merchandise imports)')| 
        (worlddf.IndicatorName == 'Food exports (% of merchandise exports)')| 
        (worlddf.IndicatorName == 'Exports of goods and services (annual % growth)')|
        (worlddf.IndicatorName == 'Exports of goods and services (% of GDP)')| 
        (worlddf.IndicatorName == 'Out-of-school children of primary school age, both sexes (number)')|
        (worlddf.IndicatorName == 'Enrolment in secondary vocational, both sexes (number)')| 
        (worlddf.IndicatorName == 'Enrolment in secondary general, both sexes (number)')|
        (worlddf.IndicatorName == 'Enrolment in secondary education, both sexes (number)')|
        (worlddf.IndicatorName == 'Enrolment in primary education, both sexes (number)')|
        (worlddf.IndicatorName == 'Labor force with tertiary education (% of total)')|
        (worlddf.IndicatorName == 'Labor force with secondary education (% of total)')|
        (worlddf.IndicatorName == 'Labor force with primary education (% of total)')|
        (worlddf.IndicatorName == 'Labor force participation rate, total (% of total population ages 15+) (national estimate)')|
        (worlddf.IndicatorName == 'Employers, total (% of employment)')|
        (worlddf.IndicatorName == 'Annualized average growth rate in per capita real survey mean consumption or income, total population (%)')|
        (worlddf.IndicatorName == 'Annualized average growth rate in per capita real survey mean consumption or income, bottom 40% of population (%)')|
        (worlddf.IndicatorName == 'Adjusted net national income per capita (annual % growth)')|
        (worlddf.IndicatorName == 'Antiretroviral therapy coverage (% of people living with HIV)')|
        (worlddf.IndicatorName == 'Adequacy of unemployment benefits and ALMP (% of total welfare of beneficiary households)')|
        (worlddf.IndicatorName == 'Adequacy of social safety net programs (% of total welfare of beneficiary households)')|
        (worlddf.IndicatorName == 'Adequacy of social protection and labor programs (% of total welfare of beneficiary households)')|
        (worlddf.IndicatorName == 'Adequacy of social insurance programs (% of total welfare of beneficiary households)')|
        (worlddf.IndicatorName == 'Improved water source, urban (% of urban population with access)')|
        (worlddf.IndicatorName == 'Improved water source, rural (% of rural population with access)')|
        (worlddf.IndicatorName == 'Improved water source (% of population with access)')|
        (worlddf.IndicatorName == 'Improved sanitation facilities, urban (% of urban population with access)')|
        (worlddf.IndicatorName == 'Improved sanitation facilities, rural (% of rural population with access)')|
        (worlddf.IndicatorName == 'Improved sanitation facilities (% of population with access)')|
        (worlddf.IndicatorName == 'Mobile cellular subscriptions (per 100 people)')|
        (worlddf.IndicatorName == 'Fixed telephone subscriptions (per 100 people)')|
        (worlddf.IndicatorName == 'Fixed broadband subscriptions (per 100 people)')|
        (worlddf.IndicatorName == 'Depth of the food deficit (kilocalories per person per day)')|
        (worlddf.IndicatorName == 'Access to electricity, urban (% of urban population)')|
        (worlddf.IndicatorName == 'Access to electricity, rural (% of rural population)')|
        (worlddf.IndicatorName == 'Access to electricity (% of population)')|
        (worlddf.IndicatorName == 'Multilateral debt (% of total external debt)')|
        (worlddf.IndicatorName == 'Life expectancy at birth, total (years)')|
        (worlddf.IndicatorName == 'Household final consumption expenditure per capita growth (annual %)')|
        (worlddf.IndicatorName == 'GDP per capita growth (annual %)')|
        (worlddf.IndicatorName == 'GDP growth (annual %)' )|
        (worlddf.IndicatorName == 'GDP per capita (current US$)' )]

In [459]:
accessdf[(accessdf['IndicatorName'] == 'GDP per capita (current US$)') & (accessdf['Country'] == 'United States')].tail()

,Country,IndicatorName,Year,Value
9441,United States,GDP per capita (current US$),2010-01-01,48374.056457
9669,United States,GDP per capita (current US$),2011-01-01,49781.357490
9894,United States,GDP per capita (current US$),2012-01-01,51456.658728
10119,United States,GDP per capita (current US$),2013-01-01,52980.043626
10335,United States,GDP per capita (current US$),2014-01-01,54629.495168


In [460]:
worlddf[(worlddf['IndicatorName'] == 'GDP per capita (current US$)') & (worlddf['Country'] == 'United States')].tail(8)

,Country,IndicatorName,Year,Value
7851928,United States,GDP per capita (current US$),2010-01-01,48466.821025
7851929,United States,GDP per capita (current US$),2011-01-01,49883.116859
7851930,United States,GDP per capita (current US$),2012-01-01,51603.496150
7851931,United States,GDP per capita (current US$),2013-01-01,53106.915497
7851932,United States,GDP per capita (current US$),2014-01-01,55032.959462
7851933,United States,GDP per capita (current US$),2015-01-01,56803.470613
7851934,United States,GDP per capita (current US$),2016-01-01,57904.204329
7851935,United States,GDP per capita (current US$),2017-01-01,59927.930286


In [461]:
len(worlddf.index), len(accessdf.index)

(175225, 227110)

In [464]:
combineddf = accessdf.merge(worlddf, on=['Country', 'IndicatorName', 'Year'], how='outer')
print(combineddf.info())
combineddf[(combineddf['IndicatorName'] == 'GDP per capita (current US$)') & (combineddf['Country'] == 'United States')].tail(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279403 entries, 0 to 279402
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Country        279403 non-null  object        
 1   IndicatorName  279403 non-null  object        
 2   Year           279403 non-null  datetime64[ns]
 3   Value_x        227110 non-null  float64       
 4   Value_y        175225 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 12.8+ MB
None


,Country,IndicatorName,Year,Value_x,Value_y
8984,United States,GDP per capita (current US$),2008-01-01,48401.427340,48401.427340
9213,United States,GDP per capita (current US$),2009-01-01,47001.555350,47001.555350
9441,United States,GDP per capita (current US$),2010-01-01,48374.056457,48466.821025
9669,United States,GDP per capita (current US$),2011-01-01,49781.357490,49883.116859
9894,United States,GDP per capita (current US$),2012-01-01,51456.658728,51603.496150
10119,United States,GDP per capita (current US$),2013-01-01,52980.043626,53106.915497
10335,United States,GDP per capita (current US$),2014-01-01,54629.495168,55032.959462
277726,United States,GDP per capita (current US$),2015-01-01,NaN,56803.470613
277727,United States,GDP per capita (current US$),2016-01-01,NaN,57904.204329
277728,United States,GDP per capita (current US$),2017-01-01,NaN,59927.930286


In [465]:
cleandf = accessdf.combine_first(worlddf)
cleandf[(cleandf['IndicatorName'] == 'GDP per capita (current US$)') & (cleandf['Country'] == 'United States')].tail(10)

,Country,IndicatorName,Year,Value
7851926,United States,GDP per capita (current US$),2008-01-01,48401.427340
7851927,United States,GDP per capita (current US$),2009-01-01,47001.555350
7851928,United States,GDP per capita (current US$),2010-01-01,48466.821025
7851929,United States,GDP per capita (current US$),2011-01-01,49883.116859
7851930,United States,GDP per capita (current US$),2012-01-01,51603.496150
7851931,United States,GDP per capita (current US$),2013-01-01,53106.915497
7851932,United States,GDP per capita (current US$),2014-01-01,55032.959462
7851933,United States,GDP per capita (current US$),2015-01-01,56803.470613
7851934,United States,GDP per capita (current US$),2016-01-01,57904.204329
7851935,United States,GDP per capita (current US$),2017-01-01,59927.930286


In [482]:
cleandf.to_csv('Data/CleanedWorld.csv')

In [483]:
happydf

,Country,Year,Happiness Rank,Happiness Score,Economy (GDP per Capita),Social support,Trust (Government Corruption),Generosity
0,Switzerland,2015-01-01,1.0,7.587,1.39651,1.34951,0.41978,0.29678
1,Iceland,2015-01-01,2.0,7.561,1.30232,1.40223,0.14145,0.43630
2,Denmark,2015-01-01,3.0,7.527,1.32548,1.36058,0.48357,0.34139
3,Norway,2015-01-01,4.0,7.522,1.45900,1.33095,0.36503,0.34699
4,Canada,2015-01-01,5.0,7.427,1.32629,1.32261,0.32957,0.45811
...,...,...,...,...,...,...,...,...
777,Rwanda,2019-01-01,152.0,3.334,0.35900,0.71100,0.41100,0.21700
778,Tanzania,2019-01-01,153.0,3.231,0.47600,0.88500,0.14700,0.27600
779,Afghanistan,2019-01-01,154.0,3.203,0.35000,0.51700,0.02500,0.15800
780,Central African Republic,2019-01-01,155.0,3.083,0.02600,0.00000,0.03500,0.23500
